In [1]:
#load data from disk
import pandas as pd
import numpy as np
import os, math
from pprint import pprint  # pretty-printer
from collections import defaultdict
from gensim.corpora.dictionary import Dictionary
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
import ast
from itertools import groupby

In [2]:
data_filepath = "../data/chengdu_taxi/"
data_filename = '20140803_sequence_full.csv'
traSequence = pd.read_csv(os.path.join(data_filepath, data_filename))
# 构造序列停留点检测
staypoint = False
def get_stay_points(df_col, staypoint):
    if staypoint == False:
        # 不加入停留点
        return df_col.apply(lambda x: [k for k, v in groupby(ast.literal_eval(x))])
    elif staypoint == True:
        # 加入停留点
        return df_col.apply(lambda x: ast.literal_eval(x))
traSequence['sequence'] = get_stay_points(traSequence['sequence'], staypoint=True)

In [3]:
# construct documents
documents = []
for seq in zip(traSequence['sequence']):
    my_list = list(seq)[0]
    my_list_pair = []
    for idx, current in enumerate(my_list):
        if len(my_list) == 1:
            my_list_pair.append(str(current) + '-' + str(current))
        elif idx < len(my_list) - 1:
            my_list_pair.append(str(current) + '-' + str(my_list[idx + 1]))
    join_s = " "
    join_s = join_s.join(my_list_pair)
    documents.append(join_s)

# construct texts
# remove words that appear only once
texts = [[word for word in document.lower().split()] for document in documents]
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text] for text in texts]

## dictionary

dictionary = Dictionary(texts)
dictionary.save('./tmp/20140803_full.dict')  # store the dictionary, for future reference
corpus = [dictionary.doc2bow(text) for text in texts]

# store the dictionary, for future reference
dict_filepath = "./tmp/"
dict_filename = 'dict-stay' + str(staypoint) + str(data_filename.split('.')[0]) + '.dict'
dictionary.save(os.path.join(dict_filepath, dict_filename))  
#corpus = [['id', 'count'], ['id', 'count']]

In [28]:
# class DTMcorpus(corpora.textcorpus.TextCorpus):

#     def get_texts(self):
#         return self.input

#     def __len__(self):
#         return len(self.input)

#     def re(self):
#         return self

# corpus = DTMcorpus(texts)


In [4]:
hour_list = np.unique(traSequence['hour'])
time_slice = [len(traSequence[traSequence['hour'] == hour]) for i, hour in enumerate(hour_list)]
model = DtmModel('../utils/dtm-win64.exe', corpus=corpus, id2word=dictionary, time_slices=time_slice, num_topics=18, initialize_lda=True)

In [ ]:
# store the model
model_filepath = "./model/"
model_filename = 'model-stay' + str(staypoint) + str(data_filename.split('.')[0])
model.save(os.path.join(model_filepath, model_filename))

# Load a potentially pre-trained model from disk.
#ldaseq = model.load(os.path.join(model_filepath, model_filename))

In [12]:
staypoint = False
str(staypoint)

'False'